In [22]:
import math
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [21]:
torch.sqrt(torch.Tensor([4]))

tensor([2.])

In [23]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        # 3 * d_model to simulate three independant matrix, we can consider these three matrices as concatenate together
        self.dkv = nn.Linear(d_model, 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask=None):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        # We create dimension for the heads to parallelize the process.
        # The last dimension contains the matrix q, k and v
        qkv = qkv.shape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        # We move the head dimension to the second position and the sequence length dimension to the third place.
        # This allows us to parallelize the calculations of the dot products K and Q for each word and then for each head.
        qkv.permute(0, 2, 1, 3)
        # We retrieve independent q, k and v matrices by chuking the qkv matrix on the last dimension
        q, k, v = qkv.chunk(3, dim=-1)
        attention_weights = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            attention_weights += mask
        attention = F

         



In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super().__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.ffn = FeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.dropout2 = nn.Dropout(p=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])


    def forward(self, x):
        residual_x = x
        x= self.attention(x, mask=None) # The encoder has to be able to look at any other word in the sentence
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x


In [ ]:
class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*(EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
                                      for _ in range(num_layers)))

    def forward(self,x):
        x = self.layers(x)
        return x

In [18]:
d_model = 512 # embedding dimension
max_length = 200 # maximum number of words for one translation
batch_size = 32 # number of "sentence" per batch
num_heads = 8 # number of heads during the self attention
drop_prob = 0.1 # probability of dropout for a better generalization
ffn_hidden = 2048 # expend 512 to 2048 during feed forward step
num_layers = 5 # number of sequential encoder

In [ ]:
encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)